# Drone Localization Pack — воспроизводимый запуск

Этот ноутбук повторяет полный пайплайн: скачивание/распаковка датасета, сбор аннотаций,
аудит, прогон моделей и агрегация результатов. Основной сценарий рассчитан на Google Colab.

In [ ]:
from pathlib import Path
import os

# Попробуем найти корень репозитория. При необходимости измените REPO_DIR вручную.
REPO_DIR = Path.cwd()
if not (REPO_DIR / 'scripts').exists():
    candidate = Path('/content/drone_localization_pack')
    if candidate.exists():
        REPO_DIR = candidate
        os.chdir(REPO_DIR)
    else:
        raise FileNotFoundError('Не найден корень репозитория. Укажите REPO_DIR вручную.')

print('Repo root:', REPO_DIR)


## Установка зависимостей

Если PyTorch уже установлен (обычно так в Colab), достаточно `requirements_base.txt`.

In [ ]:
!pip -q install -r requirements_base.txt

# Если torch/torchvision отсутствуют, установите полный набор:
# !pip -q install -r requirements_full.txt


In [ ]:
try:
    import torch
    print('torch', torch.__version__, 'cuda:', torch.cuda.is_available())
except Exception as e:
    print('Torch not ready:', e)


## Загрузка датасета из Google Drive (опционально)

Пропустите этот шаг, если датасет уже лежит в `dataset_v1/`.

In [ ]:
FOLDER_ID = '1FFvvjTX_EcXBPGPc5z2pG1y4U_nIaMvi'
!pip -q install gdown
!gdown --folder $FOLDER_ID


## Распаковка архива (Colab/Ubuntu)

In [ ]:
from pathlib import Path

# Для Colab/Ubuntu: установка p7zip и распаковка.
!apt-get update -y
!apt-get install -y p7zip-full

DATASET_ARCHIVE = Path('/content/datasets/dataset_v5.7z.001')
EXTRACT_DIR = Path('/content/extracted_dataset')
!7z x $DATASET_ARCHIVE -o$EXTRACT_DIR


## Объединение папок датасета

In [ ]:
EXTRACTED_DATASET_DIR = EXTRACT_DIR / 'dataset_v1'
TARGET_DATASET_DIR = REPO_DIR / 'dataset_v1'
!rsync -av --update $EXTRACTED_DATASET_DIR/ $TARGET_DATASET_DIR/


## Сбор аннотаций из .txt (если нужно)

In [ ]:
!python scripts/00_build_annotations_from_txt.py \
  --dataset_root dataset_v1 \
  --image_ext .png \
  --splits train val test \
  --overwrite


## Аудит датасета

In [ ]:
!python scripts/0_dataset_audit.py \
  --dataset_root dataset_v1 \
  --outdir results/dataset_audit


## ResNet18 (авто-выводность 2/3/7D)

In [ ]:
!python scripts/1_eval_resnet18_auto.py \
  --dataset_root dataset_v1 \
  --split test \
  --weights weights/best_model_18_100_128_2.pth \
  --outdir results/resnet18_2d \
  --batch_size 32 \
  --device cuda


## ResNet34 (авто-выводность 2/3/7D)

In [ ]:
!python scripts/2_eval_resnet34_auto.py \
  --dataset_root dataset_v1 \
  --split test \
  --weights weights/best_model_34_100_128.pth \
  --outdir results/resnet34_7d \
  --batch_size 32 \
  --device cuda


## ResNeXt50 (3D: lat/lon/alt)

In [ ]:
!python scripts/3_eval_resnext50_3d.py \
  --dataset_root dataset_v1 \
  --split test \
  --weights weights/best_geo_model_new.pth \
  --outdir results/resnext50_3d \
  --batch_size 16 \
  --num_workers 2 \
  --device cuda


## Агрегация результатов

In [ ]:
!python scripts/4_aggregate_results_v2.py \
  --results_root results \
  --manual manual_results.json \
  --out_xlsx results/model_comparison_v3.xlsx \
  --out_csv results/model_comparison_v3.csv
